In [ ]:
# Install dependencies
%pip install sentinelsat>=1.2.1 earthengine-api>=0.1.384 geemap>=0.30.0 rasterio>=1.3.9 geopandas>=0.14.0 matplotlib>=3.8.0 numpy>=1.26.0 pandas>=2.0.0 Pillow>=10.0.0 scikit-learn>=1.4.0 tensorflow>=2.16.0 opencv-python>=4.9.0 folium>=0.15.0 shapely>=2.0.0 pyproj>=3.6.0 tqdm>=4.66.0


In [ ]:
# Setup
import os
from datetime import datetime, timedelta
import json
from glob import glob

import ee
import geemap
import numpy as np
from PIL import Image
import rasterio

# Configuration embedded directly
REGIONS = {
    # Northern Regions - Mediterranean Coast & Nile Delta
    'alexandria': {
        'west': 29.7,
        'south': 30.9,
        'east': 30.2,
        'north': 31.4,
        'name': 'Alexandria'
    },
    'nile_delta_west': {
        'west': 30.0,
        'south': 30.3,
        'east': 31.0,
        'north': 31.5,
        'name': 'Western Nile Delta'
    },
    'nile_delta_east': {
        'west': 31.0,
        'south': 30.3,
        'east': 32.0,
        'north': 31.5,
        'name': 'Eastern Nile Delta'
    },
    'port_said': {
        'west': 32.1,
        'south': 31.0,
        'east': 32.4,
        'north': 31.3,
        'name': 'Port Said & Suez Canal North'
    },
    'damietta': {
        'west': 31.3,
        'south': 31.2,
        'east': 31.6,
        'north': 31.5,
        'name': 'Damietta'
    },
    'marsa_matruh': {
        'west': 26.5,
        'south': 31.0,
        'east': 27.5,
        'north': 31.6,
        'name': 'Marsa Matruh'
    },
    
    # Greater Cairo Region
    'cairo': {
        'west': 31.0,
        'south': 29.8,
        'east': 31.6,
        'north': 30.3,
        'name': 'Greater Cairo'
    },
    'giza': {
        'west': 30.8,
        'south': 29.7,
        'east': 31.3,
        'north': 30.1,
        'name': 'Giza & Pyramids'
    },
    'qalyubia': {
        'west': 31.1,
        'south': 30.1,
        'east': 31.5,
        'north': 30.5,
        'name': 'Qalyubia'
    },
    
    # Suez Canal Region
    'suez': {
        'west': 32.3,
        'south': 29.7,
        'east': 32.7,
        'north': 30.1,
        'name': 'Suez'
    },
    'ismailia': {
        'west': 32.1,
        'south': 30.4,
        'east': 32.5,
        'north': 30.7,
        'name': 'Ismailia'
    },
    
    # Sinai Peninsula
    'north_sinai': {
        'west': 32.8,
        'south': 30.5,
        'east': 34.3,
        'north': 31.3,
        'name': 'North Sinai'
    },
    'south_sinai': {
        'west': 33.5,
        'south': 27.7,
        'east': 34.9,
        'north': 29.5,
        'name': 'South Sinai'
    },
    'sharm_el_sheikh': {
        'west': 34.1,
        'south': 27.7,
        'east': 34.5,
        'north': 28.1,
        'name': 'Sharm El Sheikh'
    },
    
    # Red Sea Coast
    'hurghada': {
        'west': 33.5,
        'south': 26.8,
        'east': 34.0,
        'north': 27.4,
        'name': 'Hurghada'
    },
    'red_sea_north': {
        'west': 32.8,
        'south': 27.0,
        'east': 33.5,
        'north': 28.5,
        'name': 'Northern Red Sea Coast'
    },
    'red_sea_south': {
        'west': 34.0,
        'south': 23.5,
        'east': 35.5,
        'north': 25.5,
        'name': 'Southern Red Sea Coast'
    },
    'marsa_alam': {
        'west': 34.5,
        'south': 24.8,
        'east': 35.1,
        'north': 25.3,
        'name': 'Marsa Alam'
    },
    
    # Middle Egypt - Nile Valley
    'fayoum': {
        'west': 30.5,
        'south': 29.0,
        'east': 31.0,
        'north': 29.6,
        'name': 'Fayoum Oasis'
    },
    'beni_suef': {
        'west': 31.0,
        'south': 28.8,
        'east': 31.3,
        'north': 29.3,
        'name': 'Beni Suef'
    },
    'minya': {
        'west': 30.5,
        'south': 27.8,
        'east': 31.0,
        'north': 28.5,
        'name': 'Minya'
    },
    'asyut': {
        'west': 30.8,
        'south': 27.0,
        'east': 31.3,
        'north': 27.5,
        'name': 'Asyut'
    },
    'sohag': {
        'west': 31.5,
        'south': 26.3,
        'east': 32.0,
        'north': 26.8,
        'name': 'Sohag'
    },
    
    # Upper Egypt - Southern Nile Valley
    'qena': {
        'west': 32.5,
        'south': 25.8,
        'east': 33.0,
        'north': 26.3,
        'name': 'Qena'
    },
    'luxor': {
        'west': 32.5,
        'south': 25.5,
        'east': 33.0,
        'north': 25.9,
        'name': 'Luxor'
    },
    'aswan': {
        'west': 32.6,
        'south': 23.8,
        'east': 33.2,
        'north': 24.3,
        'name': 'Aswan'
    },
    'abu_simbel': {
        'west': 31.4,
        'south': 22.2,
        'east': 31.8,
        'north': 22.5,
        'name': 'Abu Simbel'
    },
    
    # Western Desert
    'western_desert_north': {
        'west': 25.0,
        'south': 29.0,
        'east': 29.0,
        'north': 31.0,
        'name': 'Northern Western Desert'
    },
    'western_desert_central': {
        'west': 25.0,
        'south': 26.0,
        'east': 29.0,
        'north': 29.0,
        'name': 'Central Western Desert'
    },
    'western_desert_south': {
        'west': 25.0,
        'south': 23.0,
        'east': 29.0,
        'north': 26.0,
        'name': 'Southern Western Desert'
    },
    'siwa_oasis': {
        'west': 25.3,
        'south': 29.0,
        'east': 25.8,
        'north': 29.4,
        'name': 'Siwa Oasis'
    },
    'bahariya_oasis': {
        'west': 28.8,
        'south': 27.7,
        'east': 29.2,
        'north': 28.1,
        'name': 'Bahariya Oasis'
    },
    'farafra_oasis': {
        'west': 27.8,
        'south': 26.8,
        'east': 28.2,
        'north': 27.2,
        'name': 'Farafra Oasis'
    },
    'dakhla_oasis': {
        'west': 28.5,
        'south': 25.3,
        'east': 29.2,
        'north': 25.8,
        'name': 'Dakhla Oasis'
    },
    'kharga_oasis': {
        'west': 30.3,
        'south': 24.8,
        'east': 30.8,
        'north': 25.5,
        'name': 'Kharga Oasis'
    },
    
    # Eastern Desert
    'eastern_desert_north': {
        'west': 31.5,
        'south': 27.5,
        'east': 33.5,
        'north': 30.0,
        'name': 'Northern Eastern Desert'
    },
    'eastern_desert_south': {
        'west': 32.5,
        'south': 24.0,
        'east': 35.0,
        'north': 27.5,
        'name': 'Southern Eastern Desert'
    }
}

SENTINEL_SETTINGS = {
    'platformname': 'Sentinel-2',
    'producttype': 'S2MSI2A',
    'cloudcoverpercentage': (0, 30)
}

DATA_DIRS = {
    'raw': 'data/raw',
    'processed': 'data/processed',
    'models': 'models',
    'results': 'results',
    'visualizations': 'visualizations'
}

PROJECT_ID = os.environ.get("GEE_PROJECT", None)

In [ ]:
# Configuration preview
print(f"Total regions configured: {len(REGIONS)}")
print(f"Sample regions: {list(REGIONS.keys())[:5]}")
print(f"Sentinel settings: {SENTINEL_SETTINGS}")


In [ ]:
# Earth Engine authentication
os.environ["GEE_PROJECT"] = "theta-decker-475418-c3"
PROJECT_ID = os.environ["GEE_PROJECT"]

try:
    ee.Initialize(project=PROJECT_ID)
    print("Earth Engine already initialized")
except Exception:
    print("Authenticating... follow the browser prompts")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print("Earth Engine initialized")


In [ ]:
# Utilities
def ensure_dirs():
    for path in DATA_DIRS.values():
        os.makedirs(path, exist_ok=True)


def init_gee(project_id=None):
    if os.path.exists('.ee_project'):
        with open('.ee_project', 'r') as fh:
            project_id = project_id or fh.read().strip()
    project_id = project_id or PROJECT_ID
    try:
        ee.Initialize(project=project_id) if project_id else ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize(project=project_id) if project_id else ee.Initialize()


def region_geometry(region):
    bounds = REGIONS[region]
    return ee.Geometry.Rectangle([
        bounds['west'],
        bounds['south'],
        bounds['east'],
        bounds['north']
    ])


In [ ]:
# Scraper helpers
def fetch_composite(region, start_date, end_date):
    geom = region_geometry(region)
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', SENTINEL_SETTINGS['cloudcoverpercentage'][1]))
        .sort('CLOUD_COVER')
    )
    if collection.size().getInfo() == 0:
        return None
    return collection.median().clip(geom)


def export_region_image(region, image, start_date, end_date, scale):
    base = f"{region}_{start_date}_{end_date}_composite"
    tif_path = os.path.join(DATA_DIRS['raw'], base + '.tif')
    geemap.ee_export_image(
        image.select(['B4', 'B3', 'B2', 'B8']),
        filename=tif_path,
        scale=scale,
        region=image.geometry(),
        file_per_band=False
    )
    return tif_path


def scrape_regions(region_list=None, days_range=90, scale=60):
    ensure_dirs()
    init_gee()
    region_list = region_list or list(REGIONS.keys())
    end_date = datetime.utcnow().date()
    start_date = end_date - timedelta(days=days_range)
    start_str, end_str = start_date.isoformat(), end_date.isoformat()
    exports = []
    for region in region_list:
        composite = fetch_composite(region, start_str, end_str)
        if composite is None:
            continue
        tif_path = export_region_image(region, composite, start_str, end_str, scale)
        exports.append(tif_path)
    return exports


In [ ]:
# Download all regions including large deserts
def download_all_regions(days_range=90):
    ensure_dirs()
    init_gee()
    
    # Skip large desert regions for standard download
    skip_regions = {
        'western_desert_north',
        'western_desert_central',
        'western_desert_south',
        'eastern_desert_north',
        'eastern_desert_south'
    }
    
    # Standard regions (60m resolution)
    standard_regions = [k for k in REGIONS.keys() if k not in skip_regions]
    desert_regions = list(skip_regions)
    
    # Date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_range)
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')
    
    print(f"Date range: {start_str} to {end_str}")
    print(f"Standard regions: {len(standard_regions)}")
    print(f"Desert regions: {len(desert_regions)}")
    
    successful = []
    failed = []
    
    # Download standard regions at 60m
    for i, region in enumerate(standard_regions, 1):
        print(f"\n[{i}/{len(standard_regions)}] Processing {region}...")
        try:
            composite = fetch_composite(region, start_str, end_str)
            if composite:
                tif_path = export_region_image(region, composite, start_str, end_str, 60)
                successful.append(region)
                
                # Auto convert to PNG
                try:
                    with rasterio.open(tif_path) as src:
                        data = src.read([1, 2, 3])
                        img = np.transpose(data, (1, 2, 0)).astype(float)
                        for band in range(3):
                            b = img[:, :, band]
                            positives = b[b > 0]
                            if positives.size:
                                p2, p98 = np.percentile(positives, (2, 98))
                                img[:, :, band] = np.clip((b - p2) / (p98 - p2), 0, 1)
                        img = (img * 255).astype(np.uint8)
                        Image.fromarray(img, mode='RGB').save(tif_path.replace('.tif', '.png'), 'PNG')
                except:
                    pass
        except Exception as e:
            print(f"Failed: {e}")
            failed.append(region)
    
    # Download large deserts at 120m or higher
    for i, region in enumerate(desert_regions, 1):
        print(f"\n[Desert {i}/{len(desert_regions)}] Processing {region} at lower resolution...")
        scale = 120
        while scale <= 480:
            try:
                composite = fetch_composite(region, start_str, end_str)
                if composite:
                    tif_path = export_region_image(region, composite, start_str, end_str, scale)
                    successful.append(region)
                    print(f"Success at {scale}m resolution")
                    break
            except Exception as e:
                if "must be less than or equal to" in str(e) and scale < 480:
                    print(f"Still too large at {scale}m, trying {scale * 2}m...")
                    scale *= 2
                else:
                    print(f"Failed: {e}")
                    failed.append(region)
                    break
    
    print(f"\n{'='*70}")
    print(f"DOWNLOAD COMPLETE")
    print(f"{'='*70}")
    print(f"Success: {len(successful)}/{len(REGIONS)}")
    print(f"Failed: {len(failed)}")
    
    return successful, failed

# Run full download
successful, failed = download_all_regions()


In [ ]:
# Convert TIFFs to PNG previews
import rasterio
import numpy as np
from PIL import Image
from glob import glob

raw_dir = DATA_DIRS['raw']
tif_files = glob(os.path.join(raw_dir, "*.tif"))
print(f"Found {len(tif_files)} TIFF file(s)")

for tif_file in tif_files:
    png_file = tif_file.replace('.tif', '.png')
    if os.path.exists(png_file):
        print(f"Skipping existing {os.path.basename(png_file)}")
        continue
    with rasterio.open(tif_file) as src:
        data = src.read([1, 2, 3])
        img = np.transpose(data, (1, 2, 0)).astype(float)
        for i in range(3):
            band = img[:, :, i]
            positives = band[band > 0]
            if positives.size == 0:
                continue
            p2, p98 = np.percentile(positives, (2, 98))
            band = np.clip(band, p2, p98)
            band = (band - p2) / (p98 - p2)
            img[:, :, i] = band
        img = (np.clip(img, 0, 1) * 255).astype(np.uint8)
        Image.fromarray(img, mode='RGB').save(png_file, 'PNG', optimize=True)
        print(f"Saved {os.path.basename(png_file)}")
